Загрузка необходимых библиотек

In [2]:
!pip install datasets
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

Загружаем датасеты

In [3]:
dataset_news = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# всего 4 класса, по 25% каждого в dataset['train']
# сформирую датасет из 8000 примеров, по 2000 примеров на каждый класс

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_short)

12000

In [5]:
import random
random.shuffle(dataset_short)
dataset_news = {'train': dataset_short[:9600], 'test': dataset_short[9600:]}

In [6]:
# Выводим объём выборок
print('Объём train:', len(dataset_news['train']))
print('Объём test:', len(dataset_news['test']))

train_texts_news = [s['news'] for s in dataset_news['train']]
test_texts_news = [s['news'] for s in dataset_news['test']]

train_labels_news = [s['label'] for s in dataset_news['train']]
test_labels_news = [s['label'] for s in dataset_news['test']]

# ВЫводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

Объём train: 9600
Объём test: 2400
New Computer Is Created Just for Teenagers This isn't your typical, humdrum, slate-colored computer. Not only is the PC known as the hip-e almost all white, but its screen and keyboard are framed in fuzzy pink fur. Or a leopard skin design. Or a graffiti-themed pattern. 3
Dim prospects for peace With public attention focused elsewhere, the Bush administration is letting Middle East peace hopes disappear. Israelis rightly complain that peace requires a partner, a role that Yasser Arafat has been pathetically incapable of filling so far. 0


In [7]:
# второй датасет
dataset_imdb = load_dataset("imdb")

count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_short_imdb)

12000

In [8]:
random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:9600], 'test': dataset_short_imdb[9600:]}

In [9]:
# Выводим объём выборок
print('Объём train:', len(dataset_imdb['train']))
print('Объём test:', len(dataset_imdb['test']))

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

print(train_texts_imdb[0])
print(train_labels_imdb[0])
print()
print(test_texts_imdb[0])
print(test_labels_imdb[0])

Объём train: 9600
Объём test: 2400
I may very well be one of the few who really stuck to this film. I also saw this movie when it came out, and I agree with the last post that Up The Acedemy was way ahead of its' time. The humor in the film itself is pure MAD magazine. I don't see why MAD stand behind this feature. It was also one of the few films of the early 80's to have a killer accompanying soundtrack with the punk and new wave bands that were emerging from L.A. at the time. I own the soundtrack and I play it constantly to this day. What can I say? There are definitely worst movies out there. I don't consider Porky's to be as funny as Up The Academy, there are some really good laughs throughout the film, and the jokes fall on either stereotypes or getting laid. Hey, nobody said this was going to be The Maltese Falcon.
1

A journey of discovery, this film follows the lives of one family living in a sleepy, island town in British Columbia. Languorous and dreamy, the inhabitants are s

In [10]:
# news example
news_example = train_texts_news[0]
print(news_example)

# imdb example
imdb_example = train_texts_imdb[0]
print(imdb_example)

New Computer Is Created Just for Teenagers This isn't your typical, humdrum, slate-colored computer. Not only is the PC known as the hip-e almost all white, but its screen and keyboard are framed in fuzzy pink fur. Or a leopard skin design. Or a graffiti-themed pattern.
I may very well be one of the few who really stuck to this film. I also saw this movie when it came out, and I agree with the last post that Up The Acedemy was way ahead of its' time. The humor in the film itself is pure MAD magazine. I don't see why MAD stand behind this feature. It was also one of the few films of the early 80's to have a killer accompanying soundtrack with the punk and new wave bands that were emerging from L.A. at the time. I own the soundtrack and I play it constantly to this day. What can I say? There are definitely worst movies out there. I don't consider Porky's to be as funny as Up The Academy, there are some really good laughs throughout the film, and the jokes fall on either stereotypes or ge

Функции предобработки

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import f1_score
stop_words = list(set(stopwords.words("english")))

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
def base_preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Zа-яА-Я ]', '', text)
    tokens = word_tokenize(text)
    return tokens

In [19]:
def stopwords_special(text):
  tokens = base_preprocessing(text)
  tokens = [word for word in tokens if word not in stop_words]  # убираем стоп-слова
  return tokens

In [21]:
def stemming(text):
  tokens = stopwords_special(text)

  stemmer = nltk.PorterStemmer()  # инициализируем стеммер
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга
  return stemmed_tokens

In [27]:
def lemmatisation(text):
  tokens = stopwords_special(text)

  lemma = nltk.WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(token) for token in tokens]
  return lemma_tokens

Векторизаторы

In [25]:
def vectorize(train_texts, test_texts, method, edit):
  if method == "count":
    vectorizer = CountVectorizer()
  else:
    vectorizer = TfidfVectorizer()

  # edit = 1 (stopwords_specoal), 2 (stemming), 3 (lemmatisation)
  if edit == 1:
    train_texts = [' '.join(stopwords_special(text)) for text in train_texts]
    test_texts = [' '.join(stopwords_special(text)) for text in test_texts]
  elif edit == 2:
    train_texts = [' '.join(stemming(text)) for text in train_texts]
    test_texts = [' '.join(stemming(text)) for text in test_texts]
  elif edit == 3:
    train_texts = [' '.join(lemmatisation(text)) for text in train_texts]
    test_texts = [' '.join(lemmatisation(text)) for text in test_texts]

  train_text = vectorizer.fit_transform(train_texts)
  test_text = vectorizer.transform(test_texts)

  return train_text, test_text

Обучение классификаторов

In [29]:
results = []
for alpha in [0.01, 0.1, 1, 10]:
  for f_p in [True, False]:
    for d in ['news', 'imdb']:
      for m in ['count', 'tfidf']:
        for edit in [1, 2, 3]:
          if d == 'news':
            train_texts = train_texts_news
            test_texts = test_texts_news
            train_labels = train_labels_news
            test_labels = test_labels_news
          else:
            train_texts = train_texts_imdb
            test_texts = test_texts_imdb
            train_labels = train_labels_imdb
            test_labels = test_labels_imdb


          train_text, test_text = vectorize(train_texts, test_texts, m, edit)

          nb_clf = MultinomialNB(alpha=alpha, fit_prior=f_p)
          nb_clf.fit(train_text, train_labels)
          nb_pred = nb_clf.predict(test_text)
          if edit == 1:
            results.append([alpha, f_p, d, m, 'stopwords_special', f1_score(test_labels, nb_pred, average='macro')])
          elif edit == 2:
            results.append([alpha, f_p, d, m, 'stemming', f1_score(test_labels, nb_pred, average='macro')])
          elif edit == 3:
            results.append([alpha, f_p, d, m, 'lemmatisation', f1_score(test_labels, nb_pred, average='macro')])

          if len(results) % 10 == 0:
            print(len(results))

10
20
30
40
50
60
70
80
90


In [31]:
results[:10]

[[0.01, True, 'news', 'count', 'stopwords_special', 0.8780675206735631],
 [0.01, True, 'news', 'count', 'stemming', 0.8843334796324799],
 [0.01, True, 'news', 'count', 'lemmatisation', 0.8793422603990133],
 [0.01, True, 'news', 'tfidf', 'stopwords_special', 0.8833754494048445],
 [0.01, True, 'news', 'tfidf', 'stemming', 0.8830513968196638],
 [0.01, True, 'news', 'tfidf', 'lemmatisation', 0.8796559113898796],
 [0.01, True, 'imdb', 'count', 'stopwords_special', 0.8465361157929489],
 [0.01, True, 'imdb', 'count', 'stemming', 0.8477517045813007],
 [0.01, True, 'imdb', 'count', 'lemmatisation', 0.8435805184054459],
 [0.01, True, 'imdb', 'tfidf', 'stopwords_special', 0.8499623864038137]]

In [32]:
df_results = pd.DataFrame(results, columns=["alpha (сглаживание)", "fit_prior (апр.вероятности)", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results.to_excel("results_nb.xlsx", index=False)

In [35]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

results_lsa = []
for k in [10, 50, 100]:
  for alpha in [0.01, 0.1, 1, 10]:
    for f_p in [True, False]:
      for d in ['news', 'imdb']:
        for m in ['count', 'tfidf']:
          for edit in [1, 2, 3]:
            if d == 'news':
              train_texts = train_texts_news
              test_texts = test_texts_news
              train_labels = train_labels_news
              test_labels = test_labels_news
            else:
              train_texts = train_texts_imdb
              test_texts = test_texts_imdb
              train_labels = train_labels_imdb
              test_labels = test_labels_imdb

            train_text, test_text = vectorize(train_texts, test_texts, m, edit)
            svd = TruncatedSVD(n_components=k, random_state=42)
            lsa = make_pipeline(svd)
            train_lsa = lsa.fit_transform(train_text)
            test_lsa = lsa.transform(test_text)

            train_lsa = np.clip(train_lsa, 0, None)
            test_lsa = np.clip(test_lsa, 0, None)

            nb_clf = MultinomialNB(alpha=alpha, fit_prior=f_p)
            nb_clf.fit(train_lsa, train_labels)
            nb_pred = nb_clf.predict(test_lsa)
            if edit == 1:
              results_lsa.append([k, alpha, f_p, d, m, 'stopwords_special', f1_score(test_labels, nb_pred, average='macro')])
            elif edit == 2:
              results_lsa.append([k, alpha, f_p, d, m, 'stemming', f1_score(test_labels, nb_pred, average='macro')])
            elif edit == 3:
              results_lsa.append([k, alpha, f_p, d, m, 'lemmatisation', f1_score(test_labels, nb_pred, average='macro')])

            if len(results_lsa) % 10 == 0:
              print(len(results_lsa))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280


In [37]:
results_lsa[:10]

[[10, 0.01, True, 'news', 'count', 'stopwords_special', 0.6223650693556702],
 [10, 0.01, True, 'news', 'count', 'stemming', 0.6539763789563652],
 [10, 0.01, True, 'news', 'count', 'lemmatisation', 0.6678196769883877],
 [10, 0.01, True, 'news', 'tfidf', 'stopwords_special', 0.6153712559371532],
 [10, 0.01, True, 'news', 'tfidf', 'stemming', 0.7379325116235838],
 [10, 0.01, True, 'news', 'tfidf', 'lemmatisation', 0.6896944164305527],
 [10, 0.01, True, 'imdb', 'count', 'stopwords_special', 0.6653998785702964],
 [10, 0.01, True, 'imdb', 'count', 'stemming', 0.6611087364715744],
 [10, 0.01, True, 'imdb', 'count', 'lemmatisation', 0.6493417078539014],
 [10, 0.01, True, 'imdb', 'tfidf', 'stopwords_special', 0.744809736833298]]

In [38]:
df_results1 = pd.DataFrame(results_lsa, columns=['k', "alpha (сглаживание)", "fit_prior (апр.вероятности)", "Dataset", "Vectorizer", "Preprocess", "F1-score"])
df_results1.to_excel("results_lsa_nb.xlsx", index=False)